In [1]:
import os
import json
import warnings

from nilearn.datasets import load_mni152_gm_mask
from nilearn import maskers
import nibabel as nib
import numpy as np
import pandas as pd
import torchio as tio
import torch

In [2]:
# load captions
with open(r"D:\Repos\clip-bold\data\intermediate\01_captions\coco_final_labels.json") as file:
    coco_captions = json.load(file)

with open(r"D:\Repos\clip-bold\data\intermediate\01_captions\imagenet_final_labels.json") as file:
    imagenet_captions = json.load(file)

with open(r"D:\Repos\clip-bold\data\intermediate\01_captions\scene_final_labels.json") as file:
    scene_captions = json.load(file)

captions = {
    "coco": coco_captions,
    "imagenet": imagenet_captions,
    "scenes": scene_captions
}

In [4]:
BOLD_DATA = r"D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep"
OUTPUT_PATH = r"D:\Repos\clip-bold\data\intermediate\02_stimulus_step"

transform = tio.Resize((50, 60, 50))

def get_nifti_files(participant: int):
    """
    Get all nii files corresponding to a participant index
    """
    data_dir = os.path.join(BOLD_DATA, f"sub-CSI{participant}")

    participant_captions = []

    # get all run data
    data_index = 0
    for root, _, files in os.walk(data_dir):
        for filename in files:
            file = os.path.join(root, filename)

            if file.endswith("_preproc.nii.gz") and "run" in file:
                print(os.path.join(root, filename))
                # extract session and run number
                session = file.split("ses-")[-1][:2]
                run = file.split("run-")[-1][:2]
                
                # get stimulus info 
                stimulus_path = rf"D:\Repos\clip-bold\data\raw\bold5000\sub-CSI{participant}\ses-{session}\func\sub-CSI{participant}_ses-{session}_task-5000scenes_run-{run}_events.tsv"
                stimulus_info = pd.read_csv(stimulus_path, sep='\t')
                
                # load BOLD data and resize
                # img = nib.load(file).get_fdata()
                img = transform(tio.ScalarImage(file))

                # first stimulus presented at 6 seconds (3 steps)
                time_index = 3
                for i in range(37):
                    # save image
                    np.save(os.path.join(OUTPUT_PATH, f"CSI{participant}-{data_index}.npy"), img.data[time_index, :, :, :].numpy())

                    # get stimulus details 
                    stimulus_type = stimulus_info.iloc[i]["ImgType"]
                    stimulus_name = stimulus_info.iloc[i]["ImgName"]
                    if "rep_" in stimulus_type:
                        stimulus_type = stimulus_type[4:]
                    # print(stimulus_type, stimulus_name)

                    # get caption
                    stimulus_caption = captions[stimulus_type][stimulus_name]
                    participant_captions.append(stimulus_caption)
                    
                    # then wait 9 seconds (5 steps)
                    time_index += 5
                    # increment index
                    data_index += 1

    with open(rf"D:\Repos\clip-bold\data\intermediate\01_captions\CSI{participant}.txt", "w") as file:
        file.writelines('\n'.join(str(i) for i in participant_captions))

with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', DeprecationWarning)
    get_nifti_files(1)

D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI1\ses-01\func\sub-CSI1_ses-01_task-5000scenes_run-01_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI1\ses-01\func\sub-CSI1_ses-01_task-5000scenes_run-02_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI1\ses-01\func\sub-CSI1_ses-01_task-5000scenes_run-03_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI1\ses-01\func\sub-CSI1_ses-01_task-5000scenes_run-04_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI1\ses-01\func\sub-CSI1_ses-01_task-5000scenes_run-05_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI1\ses-01\func\sub-CSI1_ses-01_task-5000scenes_run-06_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI1\ses-01\func\sub-CSI1_ses-01_task-5000scenes_r

In [5]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', DeprecationWarning)
    get_nifti_files(2)

D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI2\ses-01\func\sub-CSI2_ses-01_task-5000scenes_run-01_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI2\ses-01\func\sub-CSI2_ses-01_task-5000scenes_run-02_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI2\ses-01\func\sub-CSI2_ses-01_task-5000scenes_run-03_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI2\ses-01\func\sub-CSI2_ses-01_task-5000scenes_run-04_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI2\ses-01\func\sub-CSI2_ses-01_task-5000scenes_run-05_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI2\ses-01\func\sub-CSI2_ses-01_task-5000scenes_run-06_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI2\ses-01\func\sub-CSI2_ses-01_task-5000scenes_r

In [6]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', DeprecationWarning)
    get_nifti_files(3)

D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI3\ses-01\func\sub-CSI3_ses-01_task-5000scenes_run-01_bold_space-T1w_preproc.nii.gz


d:\Repos\clip-bold\.venv\Lib\site-packages\torchio\transforms\transform.py:162: RuntimeWarning: Output shape (50, 61, 50) != target shape (np.int64(50), np.int64(60), np.int64(50)). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI3\ses-01\func\sub-CSI3_ses-01_task-5000scenes_run-02_bold_space-T1w_preproc.nii.gz


d:\Repos\clip-bold\.venv\Lib\site-packages\torchio\transforms\transform.py:162: RuntimeWarning: Output shape (50, 61, 50) != target shape (np.int64(50), np.int64(60), np.int64(50)). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI3\ses-01\func\sub-CSI3_ses-01_task-5000scenes_run-03_bold_space-T1w_preproc.nii.gz


...

d:\Repos\clip-bold\.venv\Lib\site-packages\torchio\transforms\transform.py:162: RuntimeWarning: Output shape (50, 61, 50) != target shape (np.int64(50), np.int64(60), np.int64(50)). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI3\ses-15\func\sub-CSI3_ses-15_task-5000scenes_run-09_bold_space-T1w_preproc.nii.gz


d:\Repos\clip-bold\.venv\Lib\site-packages\torchio\transforms\transform.py:162: RuntimeWarning: Output shape (50, 61, 50) != target shape (np.int64(50), np.int64(60), np.int64(50)). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI3\ses-15\func\sub-CSI3_ses-15_task-5000scenes_run-10_bold_space-T1w_preproc.nii.gz


d:\Repos\clip-bold\.venv\Lib\site-packages\torchio\transforms\transform.py:162: RuntimeWarning: Output shape (50, 61, 50) != target shape (np.int64(50), np.int64(60), np.int64(50)). Fixing with CropOrPad
  transformed = self.apply_transform(subject)


In [7]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore', UserWarning)
    warnings.simplefilter('ignore', DeprecationWarning)
    get_nifti_files(4)

D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI4\ses-01\func\sub-CSI4_ses-01_task-5000scenes_run-01_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI4\ses-01\func\sub-CSI4_ses-01_task-5000scenes_run-02_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI4\ses-01\func\sub-CSI4_ses-01_task-5000scenes_run-03_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI4\ses-01\func\sub-CSI4_ses-01_task-5000scenes_run-04_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI4\ses-01\func\sub-CSI4_ses-01_task-5000scenes_run-05_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI4\ses-01\func\sub-CSI4_ses-01_task-5000scenes_run-06_bold_space-T1w_preproc.nii.gz
D:\Repos\clip-bold\data\raw\bold5000\derivatives\fmriprep\sub-CSI4\ses-01\func\sub-CSI4_ses-01_task-5000scenes_r